# An Introduction to Ibis for Python Programmers
## A More Pythonic Way To Work With Databases (Marlene Mhangami)

![](../data/elephant_with_bird.png)

A few weeks ago I was working on setting up a relational database to explore records from DataSF’s [Civic Art Collection](https://data.sfgov.org/Culture-and-Recreation/Civic-Art-Collection/r7bn-7v9c). Whenever I attend a tech conference I try to spend a day or two in the city to check out it’s cultural scene, so this seemed like useful information! I decided to use MySQL as my database engine. Coming from a Pandas background I was surprised by how unproductive and restricted I felt writing raw SQL queries. I also spent a significant amount of time resolving errors in queries that worked with one flavor of SQL but failed with MySQL. Throughout the process I kept thinking to myself if only there was a more Pythonic way!!! A few weeks later I was introduced to Ibis. 

Ibis provides a more Pythonic way of interacting with multiple database engines. 
In my own adventures I’ve always encountered Ibis’s (the bird versions) perched on top of elephants. If you’ve never seen an elephant in real life I can confirm that they are huge, complex creatures. Here’s a picture of me standing next to some for scale ;) 


![](../data/elephant_with_me.png)The image of a small bird sitting on top of a large elephant serves as a metaphor for how ibis provides a less complex, more performant way for users to interact with multiple big data engines. In fact the bigger and more complex your data the more of an argument there is to use Ibis. SQL can be quite difficult to maintain when your queries are very complex. With ibis there is no need to completely rewrite your code whether you’re scaling up or down. You can switch out your back-end if you need to and keep working in the same context but with a more powerful engine. This means your workflow is streamlined, less error prone and your cognitive load is reduced. 

At the moment, Ibis supports quite a few backends including: 

 - Traditional DBMSs: PostgreSQL, MySQL, SQLite
 - Analytical DBMSs: OmniSciDB, ClickHouse, Datafusion
 - Distributed DBMSs: Impala, PySpark, BigQuery
 - In memory analytics: pandas, Dask

Whether you’re wanting to interact with SQL databases or wanting to use distributed DBMSs, Ibis lets you do this in Python. In this post I’ll be using an SQLite backend but if you’d like to try any of the others checkout the [ibis backends](https://ibis-project.org/backends/) page for examples. Ibis currently supports over 12 backends, some of which compile to SQL and some of which are directly executed. 
![](../data/diagram.png)

For Python programmers Ibis offers a way to write SQL in Python that allows for unit-testing, composability, and abstraction over specific query engines (e.g.BigQuery)! You can carry out joins, filters and other operations on your data in a familiar, Pandas-like syntax. Overall, using Ibis simplifies your workflows, makes you more productive, and keeps your code readable.

Let's start by getting ibis and sqlite installed on your system. In the next several cells, you will notice that the first character is an exclamation point: ***!*** This tells Jupyter that we're running a shell (terminal) command instead of python code in the notebook

## Install SQLite

In [ ]:
!conda install -y -c anaconda sqlite 

## Install ibis

In [ ]:
!conda install -y -c conda-forge ibis-framework     

## Get the data
We're using the `Civic Art Collection` database, which can be downloaded here: https://data.sfgov.org/Culture-and-Recreation/Civic-Art-Collection/r7bn-7v9c using the `Export` service in the top right corner. Please select `CSV` for your export format. When you've completed the download, store the downloaded file in the `data` directory in the folder one level above.

When you list the `../data` folder you should see: 
>`Civic_Art_Collection.csv`
<br>

## Create the SQLite database
If you’d like to follow along with me, the next thing we’ll do is create a database and a table containing data about art around San Francisco from DataSF. The data is open source so you can  download the csv file directly from the site. Ibis doesn’t support loading csv files into an SQL table yet, so we’ll follow these commands to do so in the command line with SQLite

### Let's first create variables for all of the things we're about to go do

In [ ]:
import os

path_separator=os.path.sep # get the path separator (forward or backward slash) for this operating system
path=".."+path_separator+"data" #where did we store the data
file="Civic_Art_Collection.csv" #what's the name of the file we're importing
collection_name="civicArt.db" # what are we going to call the database
table_name="civicArtTable" # what are we going to call the table into which we are importing the file
db = None # connection to the database

### Create the SQL Collection (database)
SQLite wants a file with the name of the collection as the db. So, we'll use the shell `touch` command to create it. 

In [ ]:
!touch "{collection_name}"

### Create the table schema
We will use pandas to help use create the schema and, later on, load the data into SQLite

In [ ]:
import pandas as pd
type_dic = {"object": "String", "float64": "Float64", "int64": "Int64"}
art_df = pd.read_csv(path+path_separator+file).fillna(0.00)
col_names, col_types = [], []

# replace spaces in the column names with underscores
# use a dictionary to replace the column types with ibis types. 
# (We'd have to do the same thing if we were auto-coding SQL)

for e in zip(art_df.columns, art_df.dtypes):
    col_names.append('_'.join(e[0].lower().split(' ')))
    col_types.append(type_dic[e[1].__str__()])

art_df.columns = col_names


### Connecting to the database

One of the advantages of ibis is that it lets you work with tools you’re most comfortable with. I enjoy writing Python code in IPython, but you can use a jupyter notebook or any Python shell you’d like. Whatever you choose we'll use the same commands to connect to the civicArt database we created earlier. 

In [ ]:
import ibis
ibis.options.interactive = True
db = ibis.sqlite.connect(collection_name)

### Create the table
This defines where the data will go and it's 'shape' - that is, the characteristics of each column of data

In [ ]:
schema = ibis.Schema(names=col_names, types=col_types)
# if you're going through this a second time, 
# this will prevent an error trying to recreate the table definition
if not table_name in db.list_tables():
    db.create_table(table_name, schema=schema)

### import the data into the table

In [ ]:
db.load_data(table_name, art_df, if_exists='replace')

Something to note here is that pandas (read_sql)  loads data into memory and performs the computations itself. Ibis won't load the data or perform any computation. Instead it leaves the data in the database defined by the connection, and will ask the backend to perform the computations there. This means you can execute at the speed of your backend, not your local computer. A good way to think about this is to go back to our elephant metaphor. If you need some heavy lifting done, it’s probably a good idea to let an elephant do it as compared to an ibis. Our elegant ibis can pass on the heavier work to the larger database and watch as the work is done much faster than it could have done itself. 
![](../data/elephant_resting.png)
If everything went well you should be connected to the database. We’ll take a look at our data next! 
### list the tables

In [ ]:
sql_tables = db.list_tables()
print(sql_tables)

#### list the columns in the table

In [ ]:
# since the results of the preceding command is a list 
# and we may get more than one table in the list
# print each table name and the columns for that table
for each in sql_tables:
    print(f"columns in {each}:")
    print(f"{db.table(each).columns}")

## Wrangling Data With Ibis

Let's now use Ibis to carry out some common SQL commands on our dataset and find out some useful information about art in San Francisco in the process. 
### Querying 

Anything you can write in a SELECT statement you can write in Ibis. Let's test this out!
I’ll use the following code to find out which artists have art currently displayed in the city and what the title of their pieces are. 

### Selecting columns from a table 

In [ ]:
art = db.table(sql_tables[0])
sql_results = art[["artist", "display_title"]]
sql_results

Great! If you’ve used Pandas before this should feel similar to getting columns from a Pandas Dataframe, but remember the database is doing all the work for us, so this is more efficient! 
### Distinct and Count 
We can already see the length of the list, but just to show you the count method, we confirm that the number of artists that have one or more pieces on display in San Francisco is 386!

In [ ]:
distinct_artists = art["artist"].distinct()
distinct_artists.count()

### Filtering Data

Next let's pick an artist and figure out where exactly all their art is located. Adriane Colburns’ display title, `Geological Ghost` caught my eye so let's choose them! 

I use the following commands to do this

In [ ]:
adrianes_art = art.filter(art.artist == 'Colburn, Adriane')
adrianes_art["display_title", "street_address_or_intersection"]

We could do the same thing using a pandas approach to equality selection:

In [ ]:
adrianes_art = art[art.artist == 'Colburn, Adriane']
adrianes_art["display_title", "street_address_or_intersection"]

So it turns out Adriane has two pieces on display, one at `4 Guy Place` and the other at `Dagget Street & 16th Street`. This is great, we already have some places we can add to our tourist itinerary!  

### Groupby

I don’t usually stay more than one or two days in a city after a conference, so it might be nice to know which locations have the most art on display. To figure this out we’ll use the following `groupby` expressions to get the information we need. 
We use `groupy` and `sort_by` to get the 10 locations in San Francisco with the most art! 

In [ ]:
most_art = (
    art
    .filter(art.street_address_or_intersection !="0.0") # remove outliers
    .groupby("street_address_or_intersection")          # aggregate by street
    .count('display_title')
    .sort_by(ibis.desc('display_title'))                # sort by count in a descending order
    .limit(10)                                          # take top 10
)
most_art


#### Great, the first stop would be 1001 Potrero Avenue that has 59 titles on display!! 
Out of curiosity, I searched 1001 Potrero Avenue to learn more about what I expected would be an art gallery. Surprisingly it turned out to be the Zuckerberg San Francisco General Hospital. They’ve apparently invested a lot in art and have an incredible collection. 
The thing with data is that it can surprise you! The next best option is Coit tower which has its address at 1 Telegraph Hill Blvd. It is not a hospital and looks like a great place to explore! 
![](../data/coit_tower.png)
The [Coit Tower Murals](https://www.argcs.com/portfolio-item/coit-tower-murals/) were painted by Maxine Albro, Victor Arnautoff, Jane Berlandina, Ray Bertrand, Ray Boynton, Ralph Chesse, Rinaldo Cuneo, Ben Cunningham, Mallette Dean, Parker Hall, Edith Hamlin, George Harris, William Hesthal, John Langley Howard, Lucien Labaudt, Gordon Langdon, Jose Moya Del Pino, Otis Oldfield, Frederick Olmsted Jr., Ralph Stackpole, Suzanne Scheuer, Edward Terada, Frede Vidar, Clifford Wight and Bernard Zakheim.

## Joins 

Excellent. At this point we know that the next time we’re in San Francisco for a tech conference we have some great options to check out. I only found out after writing my code so let's imagine 1001 Potrero Avenue is not a hospital and use it as our first choice! Now we know where to go. What's next?

Well, oftentimes art galleries are filled with hipsters that will likely ask you about the artists on display. The good thing is that we can prepare for this in advance! Lets use an inner join to help us find which artists have the most art showing so we can look them up and feel cool by having a good response ;) 

### Use inner_join to find the most popular artists


In [ ]:
artist_location = art["street_address_or_intersection", "artist"]

artists_at_portrero = artist_location.filter(
    artist_location["street_address_or_intersection"] == "1001 Portrero Avenue").distinct()

ibis version 2.1.1 sometimes encounters an issue with views note being materialized and, when materialize, not resolving column names on a join operation. So the following code fragment is just for version 2.1.1 and uses the ```mutate()``` function (first line) to change one of the column headers and ```materialize()``` to ensure that the created view can be displayed. 

In [ ]:
artist_number_of_displays = art.mutate(c_artist=art['artist']).groupby("c_artist").count("display_title")
most_popular_artist = artists_at_portrero.inner_join(
    artist_number_of_displays,
    predicates=artists_at_portrero["artist"] == artist_number_of_displays["c_artist"]
    ).materialize()

If you're on ibis version 3.0 or later, use the following code instead to execute the join operation

In [ ]:
#artist_number_of_displays = art.groupby("artist").count("display_title")

#most_popular_artist = artists_at_portrero.inner_join(
    #artist_number_of_displays,
    #predicates=artists_at_portrero["artist"] == artist_number_of_displays["artist"]).materialize()

In [ ]:

most_popular_artist = most_popular_artist["artist", "display_title"]

most_popular_artist.sort_by(ibis.desc(most_popular_artist.display_title))


Fantastic! We now know Mary (McChesney) Fuller has the most art on display at 1001 Potrero Avenue and can search online to find out more! In my opinion we’re as ready as we’ll ever be to explore the art scene in SF! 

We learned a lot of information in a relatively short amount of time. If you’re used to writing Python code this probably feels more efficient than writing out raw SQL strings. It also means if you started your workflow in IPython or a Jupyter Notebook you can stay there. This should feel smoother and hopefully lead to increased productivity. Using tools that help you be more efficient means you have more time for creativity and exploration in your work! I wish this for you! Thanks for reading and hopefully I’ll bump into you watching elephants or in an art gallery somewhere around the world! 

